<a href="https://colab.research.google.com/github/Novadotgg/Mini-llm/blob/main/minillmSayan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install -q langchain pypdf sentence-transformers faiss-cpu gpt4all
!pip install -q langchain-community
!pip install -q transformers torch
!pip install -q transformers accelerate bitsandbytes

In [25]:
# !pip install -q langchain_community transformers torch sentence-transformers faiss-cpu pypdf langchain

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.prompts import PromptTemplate
from google.colab import files
#-------------------------------------------------------------------------------------------------------------------------------------------
# 1. Upload and process PDF
uploaded = files.upload()
pdf_name = list(uploaded.keys())[0]

loader = PyPDFLoader(pdf_name)
documents = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separator="\n"
)
texts = text_splitter.split_documents(documents)
#-------------------------------------------------------------------------------------------------------------------------------------------

# 2. Create embeddings and vector store
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

db = FAISS.from_documents(texts, embedding_model)
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1}
)
#-------------------------------------------------------------------------------------------------------------------------------------------

# 3. Setup LLM pipeline
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.3,
    do_sample=False,
    device="cpu",
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe)
#-------------------------------------------------------------------------------------------------------------------------------------------

# 4. Create QA chain with custom prompt
prompt_template = """Use the following context to answer the question.
If you don't know the answer, say you don't know. Keep the answer concise.

Context: {context}
Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)
#-------------------------------------------------------------------------------------------------------------------------------------------

# 5. Query function
def ask_question(question):
    result = qa_chain({"query": question})
    print("\nQuestion:", question)
    print("Answer:", result["result"])
    print("\nSource Page:", result['source_documents'][0].metadata['page'] + 1)
    print("Relevant Text:", result['source_documents'][0].page_content[:200] + "...")

Saving 1277.pdf to 1277.pdf


Device set to use cpu


In [27]:
print("QNA with this research paper! Type 'exit' to quit.\n")
while True:
    question = input("Your question: ")
    if question.lower() in ['exit', 'quit']:
        break
    ask_question(question)